In [100]:
# import libraries 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from scipy.stats import uniform

In [2]:
posts_df = pd.read_pickle('../data/interum/text_target.pkl')

In [3]:
posts_df.head(2)

,id,body,tags,text,target,cleaned_text,tokens
0,111102,<p>How would you explain JavaScript closures t...,javascript|scope|closures,\n\nHow would you explain JavaScript closures ...,javascript,would explain closure someone knowledge concep...,"[would, explain, closure, someone, knowledge, ..."
1,1642028,<p>After reading <a href= http://groups.google...,c++|c|operators|code-formatting|standards-comp...,\n\nAfter reading Hidden Features and Dark Cor...,c++,read hidden feature dark corner c stl comp lan...,"[read, hidden, feature, dark, corner, c, stl, ..."


In [16]:
# convert into features and target 
feature = posts_df['cleaned_text']
label = posts_df['target']

In [18]:
# split into train test
train_X, test_X, train_y, test_y = train_test_split(
    feature, label, stratify = label, test_size = 0.2, random_state = 42)

## Try CountVector

In [31]:
# try CountVector 
count_vect = CountVectorizer()
count_vect.fit(train_X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [32]:
# transform the training and test sets 
x_train_count = count_vect.transform(train_X)
x_test_count = count_vect.transform(test_X)

## Try tfidVector

In [33]:
# try tfidVector 
tfid_vect = TfidfVectorizer()
tfid_vect.fit_transform(train_X)

<24241x57072 sparse matrix of type '<class 'numpy.float64'>'
	with 949219 stored elements in Compressed Sparse Row format>

In [34]:
# transform the training and test sets 
x_train_tfidf = tfid_vect.transform(train_X)
x_test_tfidf =tfid_vect.transform(test_X)

In [14]:
print(tfid_vect.get_feature_names)

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)>


In [63]:
# initlaize all models
rf = RandomForestClassifier(random_state=42)
lg = LogisticRegression(random_state=42)
svc = SVC(random_state=42)
nb = MultinomialNB()
# xg = XGBClassifier(random_state = 42)
ada = AdaBoostClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

In [66]:
# try vanilla models 
for base_clf in (rf,lg,svc,nb,ada,gb):
        clf = OneVsRestClassifier(base_clf)
        cv_results = cross_validate(clf,x_train_tfidf,train_y, cv = 3,n_jobs=-1,return_train_score= True)
        print(base_clf)
        print('test:', cv_results['test_score'])
        print('train:', cv_results['train_score'])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
test: [0.74285361 0.74569979 0.74811239]
train: [0.9977104  0.99740099 0.99733944]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
test: [0.80076723 0.79940601 0.79848991]
train: [0.88768564 0.88935644 0.89419626]
SVC(C=1.0, cache_si

## we will focus on:

* MultinomialNB
* RandomForest
* LogisticRegression

## hyperparameter for logistic regression 

In [77]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C =uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)
clf = RandomizedSearchCV(lg, hyperparameters, random_state= 42,
                         n_iter=100, cv=5, verbose=0, n_jobs=-1)

In [79]:
best_model = clf.fit(x_train_tfidf,train_y)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [80]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 3.8950220753658367


In [97]:
clf.best_score_

0.8158079287158121

## hyperparmater for RandomForest 

In [105]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=50, stop=200, num=10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split
               }

print(random_grid)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                               n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

{'n_estimators': [50, 66, 83, 100, 116, 133, 150, 166, 183, 200], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]}


In [ ]:
rf_random.fit(x_train_tfidf, train_y)
rf_random.best_params_
rf_random.best_score_

Fitting 3 folds for each of 60 candidates, totalling 180 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 60 is smaller than n_iter=100. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


## multinominalNB 

In [ ]:
# Create regularization penalty space


# Create regularization hyperparameter distribution using uniform distribution
alpha =uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(alpha = alpha)
clf = RandomizedSearchCV(nb, hyperparameters, random_state= 42,
                         n_iter=100, cv=5, verbose=0, n_jobs=-1)

clf.fit(x_train_tfidf,train_y)

In [53]:

# parameters = {}
# grid_search = GridSearchCV(svc, param_grid={}, cv = 3, njob= -1)

In [ ]:
## Find best tfidVector using naivebaye model 


In [ ]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', lg),
])
parameters = {
        'vect_ngram_range':(1,2)
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect_min_df':(1,10),
        'vect_max_features':(100,300,500)
        'clf': (lg,),
        'clf__penalty': ('l1'),
        'clf__C': (3.895)}
grid_search = GridSearchCV(pipeline, parameters, cv = 3, return_train_score= True)